# Exploratory analysis

Incarcarea setului de date curatat anterior in notebook-ul [2_data_cleaning.ipynb](2_data_cleaning_etl.ipynb)

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [13]:
path_to_dataset = '../data/processed/2_clean_data.csv'

if not os.path.isfile(path_to_dataset):
    print('not good')

In [14]:
df = pd.read_csv(path_to_dataset)

In [15]:
df.shape

(520, 9)

In [16]:
df.head(n = 5)

,Unnamed: 0,Pret,Suprafata_Utila,Camere,Pret_mp,Zona,Vechime_Imobil,Compartimentare,Tip_Zona
0,0,72500,41.0,1,1768.29,Nicolina-CUG,Nou (Post-2000),Decomandat,Accesibil/Rezidential
1,2,180000,72.4,3,2486.19,Centru-Civic,Nou (Post-2000),Decomandat,Premium
2,11,106000,50.0,2,2120.00,Nicolina-CUG,Nou (Post-2000),Decomandat,Accesibil/Rezidential
3,12,73500,57.0,2,1289.47,Tatarasi-Tudor,Vechi (Pre-1977),Semidecomandat,Standard/Urban
4,17,105000,80.0,3,1312.50,Bucium,Nou (Post-2000),Nespecificat,Periferie


In [17]:
df['Unnamed: 0'].head(n = 2)

0    0
1    2
Name: Unnamed: 0, dtype: int64

In [18]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [20]:
df.tail(n = 5)

,Pret,Suprafata_Utila,Camere,Pret_mp,Zona,Vechime_Imobil,Compartimentare,Tip_Zona
515,112900,57.0,2,1980.70,Tatarasi-Tudor,Clasic (1977-2000),Nespecificat,Standard/Urban
516,125500,75.0,3,1673.33,Nicolina-CUG,Clasic (1977-2000),Decomandat,Accesibil/Rezidential
517,59000,35.0,1,1685.71,Nicolina-CUG,Clasic (1977-2000),Decomandat,Accesibil/Rezidential
518,127000,74.0,3,1716.22,Nicolina-CUG,Clasic (1977-2000),Nespecificat,Accesibil/Rezidential
519,90000,55.0,3,1636.36,Tatarasi-Tudor,Clasic (1977-2000),Semidecomandat,Standard/Urban


Analiza statistica a variabilelor categoriale